In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os
import sys
sys.path.append('/mnt/d/local-repo-github/enr_portfolio_modeling/')
os.chdir('D:/local-repo-github/enr_portfolio_modeling/files-storage/processed')
spark=SparkSession.builder.master('spark://pop-os.localdomain:7077').appName('blx-mdp-csv').getOrCreate()

df = spark.read.csv('D:/local-repo-github/enr_portfolio_modeling/files-storage/processed/production_asset.csv',
                   header=True,inferSchema=True)
df.show(5)
df.printSchema()

+--------+---------+---------------+-------------------+-----+-----+----+------------------+------------------+
|asset_id|projet_id|         projet|               date|année| trim|mois|           p50_adj|           p90_adj|
+--------+---------+---------------+-------------------+-----+-----+----+------------------+------------------+
|       1|     ALBE| Ally Bessadous|2022-01-01 00:00:00| 2022|Q1-22|   1|      2263.8404137|     2064.18931704|
|       2|     ALME|  Ally Mercoeur|2022-01-01 00:00:00| 2022|Q1-22|   1|1507.5778515400002|     1374.62255364|
|       3|     ALMO|   Ally Monteil|2022-01-01 00:00:00| 2022|Q1-22|   1|     1958.07452628|     1785.38933142|
|       4|     ALVE|Ally Verseilles|2022-01-01 00:00:00| 2022|Q1-22|   1|     2370.38912006|     2161.34134268|
|       5|     ART1|       Artois 1|2022-01-01 00:00:00| 2022|Q1-22|   1|3848.5022273099994|3606.6227021799996|
+--------+---------+---------------+-------------------+-----+-----+----+------------------+------------

In [ ]:
def read_spark_df(path:str, n:int)->'DataFrame':
    """Function to read csv file with spark
    Parameters
    ----------
    path : string
        path to csv file
    n : integer
        number of line to display
    Example
    -------
    >>>read_spark_df(path="D:/local-repo-github/enr_portfolio_modeling/files-storage/processed/production_asset.csv", n=10)
    """
    dataframe = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv(path).show(n)
    return dataframe

test=read_spark_df(path="D:/local-repo-github/enr_portfolio_modeling/files-storage/processed/contract_prices.csv", n=10)
df.select('projet_id', 'projet').show()

df.filter(df['année']<2023).select(['projet_id', 'projet', 'p50_adj']).show()
df.filter(df['p50_adj']>4000).show()
df.select(df['p50_adj']==2022).show()
df.filter(df['p50_adj']<2023).show()
u5000=df.filter('p50_adj>5000').collect()
u5000[0].asDict()['projet_id']
#spark sql
df.createOrReplaceTempView('prod')
df_over10000=spark.sql('select * from prod where p50_adj > 10000')
df_over10000.show()


df_over10000.describe('p50_adj').show()
df.groupBy('state').count().show()
df.agg({'age':'mean'}).show()

import pyspark.sql.functions as f
df.select(f.collect_set(df['state'])).collect()
# Returns a Row of unique states which will be all 50.
df.select(f.countDistinct('state').alias('states')).show()
#returns a single column named states with a single value of 50.
df.select(f.md5('street').alias('hash')).collect()
#Returns an md5 hash of the street value for each row
# Row(hash='81576976c4903b063c46ed9fdd140d62'),
df.select(f.reverse(df.state).alias('state-reverse')).collect()
# returns each rows street value reversed
# Row(state-reverse='nisnocsiW')
select(f.soundex(df.name).alias('soundex')).collect()
# returns a soundex of the name field for each row
# Row(soundex='P362')

spark.stop()